In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from datenspende.utils import query_ch_df, query_pg_df
import datetime
from datetime import date
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import re
import os
import glob

from functools import reduce


In [2]:
negc=pd.read_csv('neg_testdate.csv')

In [3]:
def sex(s):
    if s == 773:
        return 'female'
    elif s == 774:
        return 'male'
    else:
        return 'nd'
def get_demo(user_ids):
    
    if isinstance(user_ids, int) or isinstance(user_ids, np.int64):
        formatter = f'({user_ids})'
    elif len(user_ids) == 1:
        formatter = f'({user_ids[0]})'
    else:
        formatter = tuple(user_ids) 
 
    query = f"""
    SELECT 
        user_id, salutation, birth_date, weight, height, creation_timestamp
    FROM 
        rocs.datenspende.users
    WHERE 
        users.user_id IN {formatter} 
    """ 

    ags = query_pg_df(query)
    ags.creation_timestamp = pd.to_datetime(ags['creation_timestamp'],unit='ms') 
    ags.creation_timestamp = ags.creation_timestamp.dt.date
    ags['age'] = np.floor((2023 + 1 / 12) - ags['birth_date'] + 2.5)
    
    qu = f"""
    select    
        a.user_id,
        a.created_at,
        a.question,
        a.element        
    from 
        rocs.datenspende.answers a
    where 
        a.user_id IN {formatter}
    AND
        a.question = 127    
    """
    sxs = query_pg_df(qu)
    sxs.created_at = pd.to_datetime(sxs['created_at'],unit='ms')
    sxs.created_at = sxs.created_at.dt.date
    
    df = pd.merge(sxs, ags, how = 'outer', on = 'user_id')
    df['element'] = df['element'].apply(lambda x: sex(x))
    
    return df
def lc(x):
    if x in lcu:
        return True
    else:
        return False
value_types = pd.read_csv('epoch_value_types.csv')
value_types = value_types.rename(columns={"id": "type"})

In [4]:
dem = get_demo(negc.user_id.values)

/home/jovyan/datenspende-science/datenspende/utils/load_from_postgres.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
/home/jovyan/datenspende-science/datenspende/utils/load_from_postgres.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


In [5]:
path = os.getcwd()+'/user_epoch/'
csv_files = glob.glob(os.path.join(path, "*.csv"))

mclean = []
mcsex = []
mcage = []
 
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f, index_col = 0)
    if len(df['phase'].unique()) >= 4: #and 0 in df['phase'].unique() and 1 in df['phase'].unique():
        if np.all([np.count_nonzero(df[df['phase']==ph]['hr'].notna()) > 100 for ph in df['phase'].unique()]):
                
                 
                demo_info = f.split("/")[-1].split('.')[0]
                
                mclean.append(int(f.split("/")[-1].split('.')[1][1:]))  
                mcsex.append(re.split('(\d+)',demo_info)[0])
                mcage.append(int(re.split('(\d+)',demo_info)[1]))
                

match_user_pos = pd.DataFrame(data =  np.array([mclean, mcsex, mcage]).T,columns = ['uid', 'sex', 'age'])

In [6]:
match_user_pos['c']=1

In [7]:
match_user_pos.groupby(['age','sex']).sum()

c
age sex      
35  female  1
    male    1
40  female  2
    male    2
45  female  8
    male    2
    nd      1
50  female  5
    male    4
55  female  2
    male    2
    nd      1
60  female  3
    male    6
65  female  4
    male    3
    nd      1
70  male    1
    nd      1
75  male    1
    nd      1

In [8]:
path = os.getcwd()+'/match_neg/'
csv_files = glob.glob(os.path.join(path, "*.csv"))

 
muid = []
for f in csv_files:
    
    muid.append(int(f.split("/")[-1].split('.')[1][1:])) 
    
match_uid = np.random.choice(dem[(dem['element'] == 'male') & (dem['age'] == 70)]['user_id'].values, 30, replace=False)

match_uid = [m for m in match_uid if m not in muid]

neg_test = negc.copy()

In [9]:
def add_s(st, end):
    
    if st == end:
        end += pd.Timedelta(seconds=1)
    return end

def clean_endv(st, end):
    if end == pd.Timestamp('1970-01-01 00:00:00'):
        end = st
    return end

def get_epoch(user_ids):
     
    ft = tuple(user_ids)    
    us_data = query_ch_df(
        """SELECT * FROM rocs.vital_data_epoch WHERE vital_data_epoch.customer IN {}""".format(ft)   )
    
    us_data = pd.merge(us_data, value_types, on='type')
    us_data = us_data.drop(columns=['type'])
    us_data = us_data.rename(columns={"code": "type"})
    us_data.startTimestamp = us_data.startTimestamp//1000
    us_data.endTimestamp = us_data.endTimestamp//1000
    us_data.startTimestamp = us_data.startTimestamp.apply(lambda x: datetime.datetime.fromtimestamp(x))
    us_data.endTimestamp = us_data.endTimestamp.apply(lambda x: datetime.datetime.fromtimestamp(x))
    us_data.endTimestamp  = us_data.apply(lambda x: clean_endv(x.startTimestamp, x.endTimestamp),axis=1)
    us_data.endTimestamp  = us_data.apply(lambda x: add_s(x.startTimestamp, x.endTimestamp),axis=1)
    us_data = us_data.rename(columns={"startTimestamp": "start", "endTimestamp": "end"})
    us_data = us_data.rename(columns={"customer": "id"})
    
    return us_data


def get_as(user_ids):
    
    if isinstance(user_ids, int) or isinstance(user_ids, np.int64):
        formatter = f'({user_ids})'
    elif len(user_ids) == 1:
        formatter = f'({user_ids[0]})'
    else:
        formatter = tuple(user_ids) 
 
    query = f"""
    SELECT 
        user_id, salutation, birth_date, weight, height, creation_timestamp
    FROM 
        rocs.datenspende.users
    WHERE 
        users.user_id IN {formatter} 
    """ 

    ags = query_pg_df(query)
    ags.creation_timestamp = pd.to_datetime(ags['creation_timestamp'],unit='ms') 
    ags.creation_timestamp = ags.creation_timestamp.dt.date
    ags['age'] = np.floor((2023 + 1 / 12) - ags['birth_date'] + 2.5)
    
    qu = f"""
    select    
        a.user_id,
        a.created_at,
        a.question,
        a.element        
    from 
        rocs.datenspende.answers a
    where 
        a.user_id IN {formatter}
    AND
        a.question = 127    
    """
    sxs = query_pg_df(qu)
    sxs.created_at = pd.to_datetime(sxs['created_at'],unit='ms')
    sxs.created_at = sxs.created_at.dt.date
    
    
    if len(sxs) > 0:
        sex = 'female' if sxs['element'].values[0] == 773 else 'male'
    else: 
        sex = 'nd'
    if len(ags) > 0:
        age = ags['age'].values[0]
    else:
        age = 'nd'
    
    return sex, age

def phases(week):
    if week < 0:
        ph = 0
    elif (week >= 0 and week <= 4):
        ph = 1
    elif (week >= 5 and week <= 12):
        ph = 2
    elif week > 12:
        ph = 3
    return ph

def resample(us_data, bin_size_in_min):
           
    if len(us_data.index) > 50:
        
        user_data = us_data.copy()
        user_data = user_data[['id', 'doubleValue', 'longValue', 'booleanValue', 'start', 'end', 'source', 'type']]
        user_data.rename(
            columns={"longValue": "hr", "doubleValue": "steps", "booleanValue": "sleep"}, inplace=True
        )
        user_data["duration"] = (user_data.end - user_data.start) / pd.Timedelta(
            "1 sec"
        )
        user_data.reset_index(drop=True, inplace=True)
        
 
        df = user_data.copy()
        add_values = df[(df.duration > 60)]

        new_values = []
        for idx, row in add_values.iterrows():
            for i in np.arange(0, row.duration, 60):
                end_time = min(
                    row.end,
                    row.start
                    + pd.Timedelta("%d sec" % i)
                    + pd.Timedelta("%d sec" % 60),
                )
                new_duration = (
                    end_time - (row.start + pd.Timedelta("%d sec" % i))
                ) / pd.Timedelta("1 sec")
                new_values.append([
                    
                    row.id,
                    (row.steps / (row.duration / new_duration)),
                    row.hr,
                    row.sleep,
                    row.start + pd.Timedelta("%d sec" % i),
                    end_time,
                    row.source,
                    row.type,
                    new_duration,
                ])
                
        df = df[df.duration <= 60].append(pd.DataFrame(data=new_values, columns=user_data.columns))
        df = df.sort_values(by='start')
        df = df.groupby(['start','type']).mean().reset_index()
        
        if 'hr' not in df.columns:
            df['hr'] = np.nan
        if 'sleep' not in df.columns:
            df['sleep'] = np.nan
        
        heartrate_bin = (
            df[df.type == "HeartRate"][["start", "hr"]]
            .set_index("start")
            .resample("%d Min" % bin_size_in_min)
            .mean()
            .reset_index()
        ).dropna(subset=["hr"])
        heartrate_bin["source"] = df.source.unique()[0]
        heartrate_bin["id"] = df.id.unique()[0]
      
        restheartrate_bin = (
            df[df.type == "HeartRateRestingHourly"][["start",  "hr"]]
            .set_index("start")
            .resample("%d Min" % bin_size_in_min)
            .mean()
            .reset_index()
        ).dropna(subset=["hr"])
       
        restheartrate_bin = restheartrate_bin.rename(columns={"hr": "rhr"})
  
        sleep_bin = (
            df[df.type == "SleepStateBinary"][["start",  "sleep"]]
            .set_index("start")
            .resample("%d Min" % bin_size_in_min)
            .mean()
            .reset_index()
        ).dropna(subset=["sleep"])

        steps_bin = (
            df[df.type == "Steps"][["start",  "steps"]]
            .set_index("start")
            .resample("%d Min" % bin_size_in_min)
            .sum()
            .reset_index()
        ).dropna(subset=["steps"])     

        data_frames = [heartrate_bin, restheartrate_bin, sleep_bin, steps_bin]
        df_lc = reduce(lambda  left,right: pd.merge(left,right,on=['start'],
                                                    how='outer'), data_frames)
        return df_lc
    return 

In [10]:
NW = [144283,823064,1088725,1207820,110753,1157073,839576,1247266,1153033,1210346,1218650,882056,1143567,1225498,1208624,651047,1207387]

In [11]:
for us_id in [m for m in match_uid if m not in NW]:
    
    sex, age = get_as([us_id])

    ud = get_epoch([us_id])
    if len(ud['source'].unique()) > 1:
        # if user has multiple devices, take device with max datapoints
        d_len = []
        for sour in ud['source'].unique():
            d_len.append(len(ud[ud['source'] == sour]))    
        ud = ud[ud['source'] == ud['source'].unique()[np.argmax(d_len)]]
        user = resample(ud, 15)    
    else:
        user = resample(ud, 15)
    del ud    
    if type(user) == pd.core.frame.DataFrame:
        user['date'] = user.start.apply(lambda x: x.date())
        user['day_of_week'] = pd.to_datetime(user['date']).dt.dayofweek     

        user['weekend'] = user['day_of_week'].apply(lambda x: True if x >= 5 else False)

        user['dt'] = pd.to_datetime(neg_test['dt'][neg_test['user_id'].isin([us_id])].iloc[0])
        user['day_totest'] = pd.to_datetime(user['date']) - user['dt'] 
        user['week_totest'] = user['day_totest'].apply(lambda x: -(x.days// - 7))
        user['phase'] = user['week_totest'].apply(lambda x: phases(x))

        if sex == 'female':
            MHR = 206 - (0.88 * age)
        elif sex == 'male':
            MHR = 208 - (0.7 * age)
        else:

            MHR = np.nan

        user['% of MHR'] = (user['hr'])/ MHR * 100

        user.to_csv('match_neg/'+sex+str(age)+str(us_id)+'.csv')
        del user
     

/home/jovyan/datenspende-science/datenspende/utils/load_from_postgres.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
/home/jovyan/datenspende-science/datenspende/utils/load_from_postgres.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
/home/jovyan/datenspende-science/datenspende/utils/load_from_postgres.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
/home/jovyan/datenspende-science/datenspende/utils/load_from_pos

AttributeError: 'DataFrame' object has no attribute 'source'